# Twitter Sentiment Analysis

In [1]:
import tweepy
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
CONSUMER_API_KEY = '8fxjhgCnLZie0AjbLuntKT0EM'
CONSUMER_API_SECRET = 'd9oab3yrWQybhC64hk5YIXybhuIql20sUWvte2Z8vVQvKtM3Id'
ACCESS_TOKEN ='822833106-VKB22OHa8dXm2Hd5sEl5d5gWLmSfaH5EEmjA88DX'
ACCESS_TOKEN_SECRET = '22iR0cQfOvSQ77UFKKf4CHNLPvvNYEqzNUkugvubrLMx5'
auth = tweepy.OAuthHandler(CONSUMER_API_KEY, CONSUMER_API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

### Utility Function

In [23]:
# Function to get trending topic by country name
def get_trending_topic_by_country(country, num_trends):
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    trends_country = api.trends_available()
    for trend in trends_country:
        if trend['name'] == country:
            country_info = (trend['name'], trend['woeid'])
            print('Country: {}, WOEID: {}'.format(trend['name'], trend['woeid']))
    
    index = 1
    trending_topics = []
    trends_result = api.trends_place(country_info[1])
    for tren in trends_result[0]['trends'][:num_trends]: 
        print('#{}: {}'.format(index, tren['name']))
        trending_topics.append(tren['name'])
        index += 1
        
    return trending_topics

# Function to get tweets by keyword
def get_tweets_by_keyword(keywords, num_of_tweets, language, filename='', export_to_csv=False):
    """
    Use this function if you want export the tweet to CSV file
    keywords: tweet keyword
    num_of_tweets: number of tweets that your want to fetch
    language: tweet language
    filename: filename
    """
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    tweets = tweepy.Cursor(api.search, q=keywords, lang=language).items(num_of_tweets)
    
    # These are all I can get with free Twitter API, you can get more with the premium one
    dt_tweets = [[tweet.user.name, tweet.user.screen_name, tweet.user.location, 
                  tweet.user.verified, tweet.user.followers_count,
                  tweet.user.friends_count, tweet.user.listed_count, 
                  tweet.created_at, tweet.text, tweet.coordinates, 
                  tweet.place, tweet.favorite_count] for tweet in tweets]
    
    df_tweets = pd.DataFrame(data=dt_tweets)
    df_tweets.columns = ['username', 'screen_name', 'user_location', 'user_verified', 
                     'followers_count', 'friends_count', 'listed_count', 'tweet_created_at',
                     'tweet_text', 'tweet_coordinates', 'tweet_place', 'tweet_favorite_count']
    
    
    if export_to_csv == True:
        export_to_csv(df_tweets, filename)
        
    return df_tweets

# Overiding function 
def get_tweets_by_keyword(keywords, num_of_tweets, language):
    """
    Use this function and its parameter if you don't want export to CSV
    """
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    tweets = tweepy.Cursor(api.search, q=keywords, lang=language).items(num_of_tweets)
    
    # These are all I can get with free Twitter API, you can get more with the premium one
    dt_tweets = [[tweet.user.name, tweet.user.screen_name, tweet.user.location, 
                  tweet.user.verified, tweet.user.followers_count,
                  tweet.user.friends_count, tweet.user.listed_count, 
                  tweet.created_at, tweet.text, tweet.coordinates, 
                  tweet.place, tweet.favorite_count] for tweet in tweets]
    
    df_tweets = pd.DataFrame(data=dt_tweets)
    df_tweets.columns = ['username', 'screen_name', 'user_location', 'user_verified', 
                     'followers_count', 'friends_count', 'listed_count', 'tweet_created_at',
                     'tweet_text', 'tweet_coordinates', 'tweet_place', 'tweet_favorite_count']
        
    return df_tweets

# Function to remove unwanted string in the tweet
def remove_unwanted_string(dataframe):
    new_record = []
    tweet_text = dataframe.apply(lambda x: str(x).split())
    for record in tweet_text:
        for text in record:
            if 'RT' in text or '@' in text or '#' in text:
                record.remove(text)
        
        new_record.append(' '.join(record))
    dataframe = new_record
    return dataframe
            
# Function to print sentiments of the sentence
def sentiment_scores(sentence):
    # Create a SentimentIntensityAnalyzer object
    sid_obj = SentimentIntensityAnalyzer()
    
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary
    # which contains pos, neg, neu, and compoind scores
    sentiment_dict = sid_obj.polarity_scores(sentence)
    
    return sentiment_dict
    
# Function export to CSV
def export_to_csv(dataframe, filename):
    print('\nExported to CSV')
    return dataframe.to_csv(filename)

## Get Tweets by trending topics on spesific country

In [24]:
df_tweets_trending = get_trending_topic_by_country('Indonesia', 10)

Country: Indonesia, WOEID: 23424846
#1: Johnny
#2: #InstitutPalingBorjuis
#3: #GlobalLiveFMxBrightWin
#4: #PKS_SarangKadrun
#5: Hendri
#6: S.Pd
#7: #YOUNGJAEXDINGO
#8: #WeekendSamaShopee
#9: Klasifikasi
#10: Pee Mak


## Get Tweets by Keyword

We can pass trending topic or any keyword as a parameter to the function.

In [25]:
df_tweets = get_tweets_by_keyword(['corona', 'COVID-19'], 10, 'en')

In [26]:
df_tweets.head()

,username,screen_name,user_location,user_verified,followers_count,friends_count,listed_count,tweet_created_at,tweet_text,tweet_coordinates,tweet_place,tweet_favorite_count
0,Khalid Mehmood Awan,KhalidM13046897,,False,1623,255,2,2020-06-20 10:48:56,RT @Lahorepoliceops: News Bulletin Capital Cit...,None,None,0
1,Priyanka Yadav,Priyank58755455,,False,2,6,0,2020-06-20 10:48:32,"To overcome this Covid-19 situation, we all ha...",None,None,0
2,Clemens P. Suter,two_journeys,"Elkton, MD",False,10288,10680,190,2020-06-20 10:48:16,“The most important thing when you are running...,None,None,0
3,Apna Rajya,ApnaRajya,,False,25,13,0,2020-06-20 10:47:05,corona cases continuously increasing as covid ...,None,None,0
4,🇮🇳Arvind Joshi🇮🇳,arvind_joshi,#VijayiBharat,False,13320,4643,86,2020-06-20 10:47:04,RT @RailMinIndia: Towards a Corona Free Delhi:...,None,None,0


In [27]:
df_tweets['tweet_text'][1]

'To overcome this Covid-19 situation, we all have to follow the moto “Never give up”. Please always try to maintain… https://t.co/eWsXJsqB4e'

## Remove Unwanted String from Tweets

In [30]:
df_tweets['tweet_text'] = remove_unwanted_string(df_tweets['tweet_text'])
print(df_tweets['tweet_text'])

0    News Bulletin Capital City Police Lahore https...
1    To overcome this Covid-19 situation, we all ha...
2    “The most important thing when you are running...
3    corona cases continuously increasing as covid ...
4    Towards a Corona Free Delhi: Indian Railways h...
5    Towards a Corona Free Delhi: Indian Railways h...
6    greece is so dangerous..😷🇬🇷 if you want to liv...
7    The number of covid -19 cases are constantly i...
8    If you are too close, you can breathe in the d...
9    COVID-19 Hoax - The Crime of The Century If th...
Name: tweet_text, dtype: object


## Tweet Sentiment with VaderSentiment library/module

Note: Can not support Indonesian language

In [32]:
pos, neg, neu, comp = [], [], [], []
for text in df_tweets['tweet_text']:
    sentiment = sentiment_scores(text)
    pos.append(sentiment['pos'])
    neg.append(sentiment['neg'])
    neu.append(sentiment['neu'])
    comp.append(sentiment['compound'])
    
df_tweets_sentiment = pd.DataFrame()
# df_tweets_sentiment.columns = ['tweet_text', 'positive', 'negative', 'neutral', 'compound']
df_tweets_sentiment['tweet_text'] = df_tweets['tweet_text']
df_tweets_sentiment['positive'] = pos
df_tweets_sentiment['negative'] = neg
df_tweets_sentiment['neutral'] = neu
df_tweets_sentiment['compound'] = comp

In [36]:
df_tweets_sentiment

,tweet_text,positive,negative,neutral,compound
0,News Bulletin Capital City Police Lahore https...,0.000,0.000,1.000,0.0000
1,"To overcome this Covid-19 situation, we all ha...",0.103,0.000,0.897,0.3182
2,“The most important thing when you are running...,0.261,0.137,0.602,0.4005
3,corona cases continuously increasing as covid ...,0.000,0.000,1.000,0.0000
4,Towards a Corona Free Delhi: Indian Railways h...,0.265,0.000,0.735,0.7579
5,Towards a Corona Free Delhi: Indian Railways h...,0.265,0.000,0.735,0.7579
6,greece is so dangerous..😷🇬🇷 if you want to liv...,0.088,0.170,0.742,-0.6128
7,The number of covid -19 cases are constantly i...,0.058,0.000,0.942,0.0772
8,"If you are too close, you can breathe in the d...",0.000,0.000,1.000,0.0000
9,COVID-19 Hoax - The Crime of The Century If th...,0.101,0.176,0.723,-0.3400


Detail information about the output visit https://github.com/cjhutto/vaderSentiment